In [1]:
from model import AttentionModel

In [2]:
keyPoints = {
    'NOSE': 0,
    'LEFT_EYE': 1,
    'RIGHT_EYE': 2,
    'LEFT_EAR':3,
    'RIGHT_EAR': 4,
    'LEFT_SHOULDER': 5,
    'RIGHT_SHOULDER':6,
    'LEFT_ELBOW': 7,
    'RIGHT_ELBOW': 8,
    'LEFT_WRIST': 9,
    'RIGHT_WRIST': 10,
    'LEFT_HIP': 11,
    'RIGHT_HIP': 12,
    'LEFT_KNEE': 13,
    'RIGHT_KNEE': 14,
    'LEFT_ANKLE': 15,
    'RIGHT_ANKLE': 16,
}

In [3]:
MODEL_SLUG = '70_7_x_best'

In [4]:
OUT_PATH = f'result/model_output_video_{MODEL_SLUG}.avi' # your video path for saving
DATA_PATH = 'Анализ бригад (телефон)/Есть телефон/14_53_17.mp4' # your video path for processing
SAVE_IMG_PATH = 'result'

In [5]:
model = AttentionModel(batch_size=32)
model.load_models(f'models/{MODEL_SLUG}.pt', 'models/yolov8x-pose.pt')

In [ ]:
model.process_video(DATA_PATH, OUT_PATH, save=True)

 45%|████▌     | 2176/4803 [03:58<04:27,  9.81it/s]

In [ ]:
model.process_video(DATA_PATH, OUT_PATH, save=True)

In [ ]:
model.process_video(DATA_PATH, OUT_PATH, save=True)

In [ ]:
model.process_video(DATA_PATH, OUT_PATH, save=True)

In [ ]:
from analysis import Analyzer

In [ ]:
analyzer = Analyzer()

In [ ]:
analyzer.get_violations(pos_est_data=model.pos_est_data, detection_data=model.detected_data)

In [ ]:
model.pos_est_data.to_csv(f'result/{MODEL_SLUG}_pos_data.csv', index=0)
model.detected_data.to_csv(f'result/{MODEL_SLUG}_det_data.csv', index=0)

In [ ]:
pos_est_data = model.pos_est_data

In [ ]:
detection_data = model.detected_data

In [ ]:
analyzer.get_violations(model.pos_est_data, model.detected_data)

In [ ]:
del model

In [ ]:
import torch

torch.cuda.empty_cache()

In [ ]:
analyzer.violations

In [ ]:
model.detected_data.head(10)['Position'].apply(lambda x: len(x[0]))

In [ ]:
model.detected_data.to_csv('batchdetect.csv')

In [ ]:
model.pos_est_data.to_csv('batchestpose.csv')

mdel.detected_data

In [ ]:
detection_data.loc[1, 'Position']

In [ ]:
import numpy as np

pos_est_data = model.pos_est_data.drop_duplicates(subset=['Frame_id'])
detection_data = model.detected_data.set_index('Frame_id')
pos_est_data = pos_est_data.set_index('Frame_id')


history = []
max_allowed_dist = 1080 / 2
wrist_phone_dist = 90
violations = []


def get_center(bbox):
    return np.array([(bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2])


for frame_id in pos_est_data.index:
    persons_wrists = [pose[9:11] for pose in pos_est_data.loc[frame_id, 'KeyPoints']]
    persons_shoulders = [pose[5:7] for pose in pos_est_data.loc[frame_id, 'KeyPoints']]
    time = pos_est_data.loc[frame_id, 'Time']

    if frame_id in detection_data.index:  # we need to update phones last positions
        phones = detection_data.loc[frame_id, 'Position'].copy()
        print(type(phones))
        print(np.array(phones).shape)
        visibility = [0 for i in range(len(history))]

        if len(history) < len(phones):
            for i in range(len(history)):
                center = get_center(history[i])
                dists = list(map(lambda x: np.linalg.norm(get_center(x) - center), phones))
                min_dist = min(dists)

                if min_dist < max_allowed_dist:
                    closest_id = dists.index(min_dist)
                    history[i] = phones[closest_id]
                    visibility[i] = 1  # we can see phone on frame
                    phones.pop(closest_id)
            if phones:
                history.extend(phones)
                visibility.extend([1 for i in range(len(phones))])  # we can see all extended phones

        else:
            for j in range(len(phones)):
                center = get_center(phones[j])
                dists = list(map(lambda x: np.linalg.norm(get_center(x) - center), history))
                closest_id = np.argmin(dists)
                if dists[closest_id] < max_allowed_dist:
                    history[closest_id] = phones[j]
                    visibility[j] = 1  # we can see phone on frame
                    phones.pop(closest_id)
            if phones:
                history.extend(phones)
                visibility.extend([1 for i in range(len(phones))])  # we can see all extended phones

        #print(history, frame_id)

    # if we have info about phones pos
    if history:
        # wrist near phone
        for i in range(len(persons_wrists)):
            for wrists in persons_wrists[i]:
                dists = list(map(lambda x: np.linalg.norm(get_center(x) - wrists), history))
                min_dist = min(dists)
                closest_id = dists.index(min_dist)
                if min_dist <= wrist_phone_dist:
                    # frame, wrist pos, phone pos, index of human, phone_visibility, type of violation
                    violations.append([frame_id, time, [wrists], history[closest_id], i, visibility[closest_id]])

        # hidden phone between shoulders
        for i in range(len(history)):
            if visibility[i] == 0:  # for each invisible phone
                center = get_center(history[i])

                for j in range(len(persons_shoulders)): # for each person
                    shoulders = persons_shoulders[j]
                    wrists = persons_wrists[j]
                    if shoulders[0][0] < center[0] < shoulders[1][0] and (
                        shoulders[0][0] < wrists[0][0] < shoulders[1][0] or shoulders[0][0] < wrists[1][0] < shoulders[1][0]
                    ):  # center of phone and one of the wrist between shoulders
                        violations.append([frame_id, time, [shoulders, wrists], history[i], j, 0])


violations_processed = []
passed_time = 0
start_time = -1

count = 0
for i in range(0, len(violations) - 1):
    time_diff = violations[i + 1][1] - violations[i][1]
    print(time_diff)
    if violations[i + 1][4] == violations[i][4] and time_diff <= 1:
        if start_time == -1:
            start_time = violations[i][1]
        passed_time += time_diff

    else:
        print(passed_time, 'passed_time')
        if passed_time > 1:
            # может прерываться видимость, сделать проверку
            violations_processed.append([violations[i][4], start_time, violations[i][1], violations[i][5]])

        passed_time = 0
        start_time = -1

In [ ]:
model.pos_est_data

In [ ]:
model.detected_data

In [ ]:
model.detected_data

In [ ]:
model.detected_data.Position[0]

In [ ]:
model.detected_data.to_csv('a.csv')

In [ ]:
import pandas as pd

df = pd.read_csv('a.csv')

In [ ]:
df.loc[1, 'Position']

In [ ]:
model.detected_data.loc[1, 'Position'][0][0]

In [ ]:
model.detected_values

In [ ]:
model.detected_data.to_csv('result/phones.csv')
model.pos_est_data.to_csv('result/keypoints.csv')

In [ ]:
model.detected_data.head()

In [ ]:
dropeddf = model.pos_est_data.drop_duplicates(subset=['Frame_id', 'Time', "People_count"])

In [ ]:
len(dropeddf)

In [ ]:
dropeddf.head(10)

In [ ]:
model.detected_data.tail()

In [ ]:
yolodf = model.detected_data

In [ ]:
yolodf = yolodf.set_index('Frame_id')

In [ ]:
yolodf.head()

In [ ]:
import pandas as pd

yolodf = pd.read_csv('yolo.csv', index_col=0)
yolodf

In [ ]:
dropeddf = pd.read_csv('posest.csv', index_col=0)
dropeddf

In [ ]:
yolodf

In [ ]:
dropeddf = dropeddf.set_index('Frame_id')


In [ ]:
dropeddf.head()

In [ ]:
[bbox['cell phones'] for bbox in yolodf.loc[1, 'Position'].values]

In [ ]:
wrists = [pose[9:11] for pose in dropeddf.loc[1, 'KeyPoints']]

In [ ]:
wrists = [pose[9:11] for pose in dropeddf.loc[1, 'KeyPoints']]
wrists

In [ ]:
def get_center(bbox):
    return [(bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2]

In [ ]:
phones = [bbox['cell phones'] for bbox in yolodf.loc[1, 'Position']]

In [ ]:
history = []
for frame_id in dropeddf.index:
    if frame_id in yolodf.index:
        phones = [bbox['cell phones'] for bbox in yolodf.loc[frame_id, 'Position'].values]
        wrists = [pose[9:11] for pose in dropeddf.loc[frame_id, 'KeyPoints']]
        # iter_arr = history if len(history) < len(phones) else phones
        if len(history) < len(phones):
            for bbox in history:
                bbox_center = get_center(bbox)

                bbox[]
            

In [ ]:
import numpy as np

(dropeddf.Frame_id.values != np.arange(1, len(dropeddf.Frame_id.values) + 1)).sum()

In [ ]:
dropeddf.to_csv('posest.csv')

In [ ]:
yolodf.to_csv('yolo.csv')

In [ ]:
dropeddf.tail(10)

In [ ]:
(dropeddf.index.values != dropeddf['Frame_id'].values - 1).sum()

In [ ]:
import pandas as pd

pos_est_data = pd.DataFrame(model.pos_est_values, columns=['Frame_id', 'Time', *model.keypoints])
pos_est_data

In [ ]:
pos_est_data

In [ ]:
model.pos_est_data.head(10)

In [ ]:
model.pos_est_data.info()

In [ ]:
from ultralytics import YOLO

model = YOLO('models/yolov8x-pose.pt').to('cuda')
result = model.predict(DATA_PATH, stream=True, save=True)

In [ ]:
model('result/model_output_video.avi')

In [ ]:
model.pos_est_data.to_csv('result/pos_est_data.csv', index=0)

In [ ]:
model.detected_data.to_csv('result/detected_data.csv', index=0)